# **Reinforcement Learning: An Introduction** - Chapter 7
Thomas Hopkins

## **Exercise 7.1**
The larger random walk task was most likely used because it would allow for larger $n$ steps to be taken and tested. A smaller walk would most likely shift the value of $n$ down. The change from the left-side end state from $0$ to $-1$ would cause larger $n$ to be more useful since a reward other than $0$ is computed along the $n$-step return if it reaches that terminal state.

## **Exercise 7.2**
On-line methods most likely worked better than off-line methods because better estimates for values were used earlier in the learning process. Waiting until the end of the episode delays the ability to use the most up-to-date information. On the other hand, on-line methods allow this new information to be used immediately.

## **Exercise 7.3**
The only explanation I can think of is that 3-step returns are particularly noisy for this choice of initialization and number of states in the random walk. So, using a higher learning rate, $\alpha$, causes the error to increase sharply over the first 10 episodes.

## **Exercise 7.4**
The equation for the weighting at time $t$ given by $\lambda$ and its half-life $\tau_{\lambda}$ is
$$(1 - \lambda)\lambda^t = (1 - \lambda)(\frac{1}{2})^{t/\tau_{\lambda}}$$
$$\Rightarrow \lambda^t = (\frac{1}{2})^{t/\tau_{\lambda}}$$
$$\Rightarrow t\ln(\lambda) = \frac{t}{\tau_{\lambda}}\ln( \frac{1}{2})$$
$$\Rightarrow \tau_{\lambda} = -\frac{\ln(2)}{\ln(\lambda)}$$